# XOR

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
s = json.load( open("styles/bmh_matplotlibrc.json") )
matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]
from ipywidgets import interact, interact_manual, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import random

In [2]:
from mpl_toolkits.mplot3d import Axes3D

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [3]:
import numpy as np

## Función de activación

In [4]:
def logistica(z):
    """
    Devuelve la función logística evaluada componente por componente
    """
    return 1 / (1 + np.exp(-z))

In [5]:
from matplotlib.pyplot import figure

@interact()
def plot_log():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma$")
    plt.title("$\sigma = \\frac{1}{1 + e^{-z}}$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [6]:
def derivada_logistica(z):
    """
    Función que, dado un arreglo de valores z
    calcula el valor de la derivada para cada entrada.
    """
    g = logistica(z)
    return g * (1 - g)

In [7]:
@interact
def plot_logp():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [8]:
def derivada_logistica_atajo(val_sigma):
    """
    Función que, dado un arreglo de valores de sigma(z)
    calcula el valor de la derivada para cada entrada.
    
    Si ya se cuenta con esos valores, es más eficiente
    calcular esto directamente.
    """
    return val_sigma*(1-val_sigma)
    ## TODO

In [10]:
@interact
def plot_logpa():
    z = np.arange(-5,5,0.1)
    val = logistica(z)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica_atajo(val))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [11]:
np.random.seed(10)

## Función de error: Entropía cruzada

La función de error más sencilla utilizada para problemas de clasificación es la **entropía cruzada**
\begin{align}
  J(\Theta) =& - \frac{1}{m} \left[ \sum_{i=1}^m \sum_{k=1}^K   y_k^{(i)} \log(h_\Theta(x^{(i)}))_k  + (1 - y_k^{(i)}) \log(1 - h_\Theta(x^{(i)}))_k   \right]    +  \\
  & \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_L} \sum_{j=1}^{s_{l+1}} (\theta_{ji}^{(l)})^2
\end{align}
donde:
  * $K$ es el número de neuronas de salida.
  * $s_l$ es el número de neuronas en la capa $l$.
  * $m$ es el número de ejemplares de entrenamiento.
  
En su forma vectorizada, las componentes del gradiente están dadas por:
\begin{align}
  \delta^{(L)} &= (A^{(L)})^T - Y  \\
  \delta^{(L-1)} &= \delta^{(L)} \Theta_{[:,1:]}^{(L-1)} \circ g'(z^{(L-1)}) \\
  ... \\
  \delta^{(1)} &= \delta^{(0)} \Theta_{[:,1:]}^{(1)} \circ g'(z^{(1)}) \\
\end{align}
con:
\begin{align}
  g'(z^{(l)}) = A^{(l)} \circ (1 - A^{(l)})
\end{align}
en general:
\begin{align}
  \Delta^{(l)} =& (\delta^{(l+1)})^T A^{(l)}   &   \nabla^{(l)} =& \frac{1}{m}\Delta^{(l)}
\end{align}

In [12]:
def cross_entropy(y, h):
    return - y * np.log(h) - (1 - y) * np.log(1 - h)

In [13]:
@interact
def plot_crossentropy():
    """
    Muestra como, en el rango donde está definida, la entropía cruzada tiene
    sus valores más pequeños donde 'y' y 'h' coinciden, y su valor más alto
    donde éstos son opuestos.
    """
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    # Datos
    X = np.arange(0.01, 1, 0.05)
    Y = np.arange(0.01, 1, 0.05)
    X, Y = np.meshgrid(X, Y)
    CE = cross_entropy(X, Y)
    
    surf = ax.plot_surface(X, Y, CE, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_xlabel("$y$")
    ax.set_ylabel("$h$")
    ax.set_title("Entropía cruzada")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Red neuronal
La red implementa encadenamiento hacia adelante (para evaluar) y hacia atrás (para entrenarse).

In [14]:
# Cuando se programa un algoritmo que depende de la generación de números aleatorios
# es buena costumbre fijar la semilla a partir de la cual dichos números son generados.
# De este modo el comportamiento sobre los datos de entrada será predecible mientras
# se está desarrollando/probando el código.
#
# Una vez el código está listo, se debe usar variando los valores de esta semilla
# para ver los efectos de la aleatoriedad simulada.

np.random.seed(10)
random.seed(10)

In [15]:
def productomatrix(x,y):
    p=[]
    for i in x:
        x_1=[]
        for j in y:
            x_1.append(np.dot(i,j))
        p.append(x_1)
    return p 

In [16]:
class Multicapa:
    """
    Red neuronal con tres capas:
    
    Entrada
    Oculta
    Salida
    
    Los parámetros (pesos) que conectan las capas se encuentran en matrices
    con los nombres siguientes:
    
    Entrada -> self.Theta_0 -> Oculta
    Oculta  -> self.Theta_1 -> Salida
    """
    def __init__(self, n_entrada, n_ocultas, n_salidas):
        """
        Inicializa la red neuronal, con pesos Theta_0 y Theta_1 aleatorios,
        esta implementación debe incluir el uso de sesgos, por lo que éstos
        no se cuentan en los parámetros siguientes, puedes incluirlos como
        neuronas extra o en sus propias matrices, sólo sé consistente pues
        esto afectará tu implementación.
        
        :param n_entrada: número de datos de entrada (sin contar el sesgo)
        :param n_ocultas: número de neuronas ocultas
        :param n_salidas: número de nueronas de salida
        """
        random.seed(0)
        self.entradas=n_entrada
        self.ocultas=n_ocultas
        self.salidas=n_salidas
        W1=[]
        for j in range(self.ocultas):
            w1=[]
            i=0
            while(i<self.entradas+1):
                w1.append(random.uniform(-0.5,0.5))
                i+=1
            W1.append(w1)  
        W1=np.array(W1)    
        self.Theta_0=W1
        W2=[]
        for j in range(self.salidas):
            w2=[]
            i=0
            while(i<self.ocultas+1):
                w2.append(random.uniform(-0.5,0.5))
                i+=1
                
            W2.append(w2)
        W2=np.array(W2)    
        self.Theta_1=W2
    def feed_forward(self, X, vector = None):
        """ Calcula las salidas, dados los datos de entrada en forma de matriz.
        Guarda los parámetros siguientes:
        A0: activaciones de la capa de entrada, ya con sesgos
        Z1: potenciales de la capa oculta, aún sin sesgo
        A1: activaciones de la capa oculta, ya con sesgos
        Z2: potenciales de la capa de salida
        A2: activaciones de la capa de salida
        
        :param vector: [opcional] se utilizarán los pesos indicados en este
                       vector en lugar de los pesos actuales de la red.
        """
        ## TODO
        X_1=X.tolist()
        for i in range(len(X)):
            X_1[i].insert(0,1)
        A0=np.array(X_1)    
        Z1=np.array(productomatrix(A0,self.Theta_0))
        A1=logistica(Z1)
        A1=A1.tolist()
        for i in range(len(A1)):
            A1[i].insert(0,1)
        A1=np.array(A1)    
        Z2=np.array(productomatrix(A1,self.Theta_1))
        A2=logistica(Z2)
        self.A0=A0
        self.A1=A1
        self.A2=A2
    def back_propagate(self, X, Y, lambda_r = 0.0):
        """ Calcula el error y su gradiente dados los pesos actuales de la red
        y los resultados esperados.
        
        Guarda el error en el atributo self.error y el gradiente en matrices
        self.Grad_1 y self.Grad_0, que tienen la misma forma de Theta_0 y Theta_1.
        
        :param X: matriz de entradas
        :param Y: matriz de salidas deseadas
        :param lambda_r: coeficiente de regularización
        """
        ## TODO
        J=0
        J_theta= - np.sum(np.dot(Y.T, np.log(self.A2)) + np.dot((1 - Y).T, np.log(1 - self.A2))) / len(X)
        the=np.array(self.vector_weights())
        theta_sum=np.dot(the.T,the)[0][0]
        Delta_2=(Y-self.A2)
        Grad_1=-(1/len(X))*np.dot(self.A1.T,Delta_2)
        gz_1=self.A1[:,1:]*(1-self.A1[:,1:])
        Delta_1=np.dot(Delta_2,self.Theta_1[:,1:])*gz_1
        Grad0=-(1/len(X))*np.dot(xor.A0[:,1:].T,Delta_1)
        Grad0_b=- np.sum(Delta_1, axis=0, keepdims=True) / len(X)
        Grad0=Grad0.T.tolist()
        for i in range(len(self.Theta_0)):
            Grad0[i].insert(0,Grad0_b[0,i])
        Grad_0=np.array(Grad0)
        k=J_theta+((lambda_r)/(2*(len(X))))*theta_sum
        self.error=k
        self.Grad_1=Grad_1.T
        self.Grad_0=Grad_0 
    def calc_error(self, X, Y, vector, lambda_r = 0.0):
        """
        Calcula el error que se cometería utilizando los pesos en 'vector' en lugar
        de los pesos actuales de la red.
        
        :returns: el error
        """
        ## TODO
    
    def vector_weights(self):
        """
        Acomoda a todos los parámetros en las matrices de sesgos y pesos, en un solo vector.
        
        :returns: vector de parámetros
        """
        VP=[]
        for i in self.Theta_0:
            for j in i:
                VP.append([j])
        for i in self.Theta_1:
            for j in i:
                VP.append([j])
        return VP
        ## TODO
    
    def reconstruct_matrices(self, vector):
        """
        Dado un vector, rearma matrices del tamaño de las matrices de sesgos y pesos.
        
        :returns: matrices de parámetros
        """
        ## TODO
        t1=[]
        theta_1=[]
        theta_0=[]
        for i in range(self.ocultas):
            t0=[]
            for j in range(self.entradas+1):
                t0.append(vector[j+i*(self.entradas+1)][0])
            theta_0.append(t0)  
        for i in range(self.ocultas+1):
            t1.append(vector[i+2*(self.entradas+1)][0])
        theta_1.append(t1)  
        return theta_0,theta_1
    def approx_gradient(self, X, Y, lambda_r = 0.0):
        """
        Aproxima el valor del gradiente alrededor de los pesos actuales,
        perturbando cada peso, uno por uno hasta estimar la variación alrededor
        de cada peso.
        
        En este método se itera sobre cada peso w:
        * Sea w - epsilon -> val1, se calcula el error e1 cometido por la red si w es
                                   reemplazado por val1.
        * Sea w + epsilon -> val2, se calcula el error e2 cometido por la red si w es
                                   reemplazado por val2.
        * La parcial correspondiente se estima como (val1 - val2)/(2 * epsilon)
        
        Este método sólo se utiliza para verificar que backpropagation esté bien
        implementado, ya que en la práctica es muy lento y menos preciso.
        
        :returns: matrices que tienen la misma forma de Theta_0 y Theta_1, donde
                  cada entrada es la estimación de la parcial del error con
                  respecto al peso correspondiente
        """
        ## TODO
        g_params = [None, None]
        epsilon = 0.0001
        W=[self.Theta_0,self.Theta_1]
        for t in range(len(W)):
            param=W[t]
            g_param = np.zeros(param.shape)
            for i in range(param.shape[0]):
                for j in range(param.shape[1]):
                    temp = param[i,j]

                    param[i,j] = temp - epsilon
                    self.feed_forward(X)
                    val1 = - np.sum(np.dot(Y.T, np.log(self.A2)) + np.dot((1 - Y).T, np.log(1 - self.A2))) / len(X)

                    param[i,j] = temp + epsilon
                    self.feed_forward(X)
                    val2 = - np.sum(np.dot(Y.T, np.log(self.A2)) + np.dot((1 - Y).T, np.log(1 - self.A2))) / len(X)

                    g_param[i,j] = (val2 - val1) / (2 * epsilon)
                    param[i,j] = temp
                g_params[t] = g_param
        self.approx_0=g_params[0]
        self.approx_1=g_params[1]
        #return approx_0,approx_1      
        
    def gradient_descent(self, X, Y, alpha, ciclos=10, check_gradient = False, lambda_r = 0.0):
        """ Evalúa y ajusta los pesos de la red, de acuerdo a los datos en X y los resultados
        deseados en Y.  Al final grafica el error vs ciclo.  Si el entrenamiento es correcto
        el error debe descender por cada iteración (ciclo).
        
        :param X: datos de entrada
        :param Y: salidas deseadas
        :param alpha: taza de aprendizaje
        :param ciclos: número de veces que se realizarán ajustes para todo el conjunto de datos X
        :param check_gradient: se calculará el gradiente con backpropagation y con aproximación por
                               perturbaciones, imprimiendo los valores lado a lado para que puedan
                               ser comparados.
        :param lambda_r: coeficiente de regularización
        """
        Z=[]
        for i in range(ciclos):
            self.feed_forward(X)
            self.back_propagate(X,Y,lambda_r)
            self.Theta_0=self.Theta_0-(alpha*(self.Grad_0))
            self.Theta_1=self.Theta_1-(alpha*(self.Grad_1))
            Z.append(self.error)
        self.errores=Z    
        if(check_gradient):
            self.Theta_0=self.Theta_0+(alpha*(self.Grad_0))
            self.Theta_1=self.Theta_1+(alpha*(self.Grad_1))
            self.approx_gradient(X,Y)
            print("Grad_0= ",self.Grad_0)
            print("Approx_0= ",self.approx_0)
            dif_1=self.Grad_0-self.approx_0
            print("Diff= ",dif_1)
            print("Grad_1= ",self.Grad_1)
            print("Approx_1= ",self.approx_1)
            dif_2=self.Grad_1-self.approx_1
            print("Diff= ",dif_2)
        x = np.arange(ciclos)
        if ciclos > 1:
            plt.title("Error")
            plt.xlabel("ciclo")
            plt.scatter(x, self.errores)    
        return self.error
    def print_output(self):
        """
        Muestra en pantalla los valores de salida obtenidos en la última ejecución de feed_forward.
        """
        O=X.tolist()
        for i in range(len(self.A2)):
            O[i].append(self.A2[i][0])
        O=np.array(O)
        print(O)
        ## TODO

# Conjunto de datos

Probaremos esta red con la función más sencilla que requiere más de un perceptrón: $XOR$.

Para ello requerimos una red con la arquitectura siguiente:

<img src="figuras/xor_sin_pesos.png">

In [17]:
# Datos
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Entradas
Y = np.array([[0], [1], [1], [0]])              # Salidas deseadas

# Instanciación de la red
xor = Multicapa(2, 2, 1)

# Probamos qué valores calcula con una inicialización aleatoria
xor.feed_forward(X)
xor.print_output()

[[0.         0.         0.53951412]
 [0.         1.         0.54059727]
 [1.         0.         0.53651897]
 [1.         1.         0.5375526 ]]


Para verificar si la red funciona para lo que fue diseñada, no basta con reducir el error,
debemos evaluar si calcula la función que queremos que calcule.  En este caso, queremos
usarla para evaluar una función binaria.  Si realizamos un redondeo, veamos cuántas respuestas
calcula correctamente.

In [18]:
xor.back_propagate(X,Y)

In [19]:
def count_correct(red, X, Y):
    """ Conciderando que las etiquetas en Y son binarias (0 y 1)
    la salida h de la red se tomará como 0 si h < 0.5
    y como 1 si h >= 0.5
    """
    red.feed_forward(X)
    H = red.A2                        # Ojo: dependiendo de tu implementación, podrías no querer la .T
    H = np.rint(H)
    print("H = ", H)
    print("Y = ", Y)
    c = np.count_nonzero((H - Y)==0)
    print("Se calcularon correctamente ", c, "entradas.")

In [20]:
c = count_correct(xor, X, Y)

H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


Para poder implementar el algoritmo de optimización, las variables a optimizar, es decir, los pesos, deben ser colocados sobre un vector (o matriz columna, en este caso).  Es necesario convertir las matrices de pesos a vector y, para usar la red, revertir el proceso, reconstruyendo las matrices a partir del vector con los pesos.

In [21]:
# Con esta casilla verifica que tus conversiones sean correctas

print("Valores actuales de los pesos")

print("Theta_0:\n", xor.Theta_0, "\n\nTheta_1:\n", xor.Theta_1)

print("\nMatrices a vector de pesos: ")
print(xor.vector_weights())

print("\nReconstrucción de las matrices a partir del vector de pesos: ")
T0, T1 = xor.reconstruct_matrices(xor.vector_weights())
print(T0, T1)

Valores actuales de los pesos
Theta_0:
 [[ 0.34442185  0.2579544  -0.07942842]
 [-0.24108325  0.01127472 -0.09506586]] 

Theta_1:
 [[ 0.28379859 -0.19668727 -0.02340305]]

Matrices a vector de pesos: 
[[0.3444218515250481], [0.2579544029403025], [-0.079428419169155], [-0.24108324970703665], [0.01127472136860852], [-0.09506586254958571], [0.2837985890347726], [-0.19668727392107255], [-0.02340304584764419]]

Reconstrucción de las matrices a partir del vector de pesos: 
[[0.3444218515250481, 0.2579544029403025, -0.079428419169155], [-0.24108324970703665, 0.01127472136860852, -0.09506586254958571]] [[0.2837985890347726, -0.19668727392107255, -0.02340304584764419]]


In [22]:
# Con esta casilla revisa que backpropagation y aproximación por perturbaciones
# den resultados semejantes.  Observa que se ejecuta sobre un solo ciclo pues es lento.

xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True)

Grad_0=  [[-1.85565376e-03 -9.65719189e-04 -6.26720898e-04]
 [-2.21281709e-04 -9.68293430e-05 -1.12552638e-04]]
Approx_0=  [[-1.85565376e-03 -9.65719189e-04 -6.26720898e-04]
 [-2.21281708e-04 -9.68293423e-05 -1.12552638e-04]]
Diff=  [[-8.24373556e-13 -6.26520537e-13  3.98233529e-13]
 [-1.24696694e-12 -6.62764006e-13 -1.18218694e-13]]
Grad_1=  [[0.03854574 0.02344828 0.01655268]]
Approx_1=  [[0.03854574 0.02344828 0.01655268]]
Diff=  [[3.22165905e-11 7.70768172e-12 2.47806636e-12]]


0.6961073967473654

In [23]:
@interact_manual(ciclos = (50, 2000))
def train_XOR(ciclos):
    xor.gradient_descent(X, Y, 0.3, ciclos)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [24]:
# Después de haber sido correctamente entrenada, la red debe producir salidas
# cercanas a las deseadas.  Dado que la función de activación es la sigmoide
# nunca llegará a 0 o 1 exactamente.

xor.feed_forward(X)
xor.print_output()

print("\nTheta_0 = ", xor.Theta_0, "\n\nTheta_1", xor.Theta_1)

[[0.         0.         0.53951412]
 [0.         1.         0.54059727]
 [1.         0.         0.53651897]
 [1.         1.         0.5375526 ]]

Theta_0 =  [[ 0.34442185  0.2579544  -0.07942842]
 [-0.24108325  0.01127472 -0.09506586]] 

Theta_1 [[ 0.28379859 -0.19668727 -0.02340305]]


In [25]:
c = count_correct(xor, X, Y)

H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


## Regularización

In [26]:
xor_reg = Multicapa(2, 2, 1)

In [27]:
c = count_correct(xor_reg, X, Y)

H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


In [28]:
xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True, lambda_r = 0.15)

Grad_0=  [[-1.85565376e-03 -9.65719189e-04 -6.26720898e-04]
 [-2.21281709e-04 -9.68293430e-05 -1.12552638e-04]]
Approx_0=  [[-1.85565376e-03 -9.65719189e-04 -6.26720898e-04]
 [-2.21281708e-04 -9.68293423e-05 -1.12552638e-04]]
Diff=  [[-8.24373556e-13 -6.26520537e-13  3.98233529e-13]
 [-1.24696694e-12 -6.62764006e-13 -1.18218694e-13]]
Grad_1=  [[0.03854574 0.02344828 0.01655268]]
Approx_1=  [[0.03854574 0.02344828 0.01655268]]
Diff=  [[3.22165905e-11 7.70768172e-12 2.47806636e-12]]


0.7032049609194603

In [29]:
@interact_manual(ciclos = (50, 2000))
def train_XOR_reg(ciclos):
    # Prueba diferentes valores de lambda_r ¿qué tanto lo puedes incrementar si matar a la red?
    xor_reg.gradient_descent(X, Y, 0.3, ciclos, lambda_r = 0.005)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [33]:
xor_reg.feed_forward(X)
xor_reg.print_output()
print("Theta_0 = ", xor_reg.Theta_0, "\nTheta_1", xor_reg.Theta_1)

[[0.         0.         0.04876228]
 [0.         1.         0.94393514]
 [1.         0.         0.95123573]
 [1.         1.         0.0407789 ]]
Theta_0 =  [[ 2.97929354  5.75657305 -5.64874085]
 [-2.44817156  4.59372451 -4.90189787]] 
Theta_1 [[ 3.28646135 -7.22008093  7.71092354]]


In [34]:
c = count_correct(xor_reg, X, Y)

H =  [[0.]
 [1.]
 [1.]
 [0.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  4 entradas.


In [32]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()